In [1]:
from dqn.agent import CarRacingDQN
import os
import tensorflow as tf
import gym
import _thread
import re
import sys

import h5py

from matplotlib import pyplot as plt

import numpy as np

from datetime import datetime as dt

#from car_racing import CarRacingFixed#
from car_racing_ellipse import CarRacingEllipse
#from car_racing_square 

model_config = dict(
    min_epsilon=0.1,
    max_negative_rewards=12,
    min_experience_size=int(1e4),
    num_frame_stack=1,
    frame_skip=3,
    train_freq=4,
    batchsize=64,
    epsilon_decay_steps=int(1e5),
    network_update_freq=int(1e3),
    experience_capacity=int(4e4),
    gamma=0.95
)

#env_name = "CarRacing-v0"
#env = gym.make(env_name)

env = CarRacingEllipse()

# tf.reset_default_graph()
dqn_agent = CarRacingDQN(env=env, **model_config)
dqn_agent.build_graph()
sess = tf.InteractiveSession()
dqn_agent.session = sess

saver = tf.train.Saver(max_to_keep=100)

#checkpoint_path = "data/checkpoint02"
checkpoint_path = '/home/dahlin/Desktop/Unpack/Car Racing Ellipse/Checkpoints'

print("loading the latest checkpoint from %s" % checkpoint_path)
ckpt = tf.train.get_checkpoint_state(checkpoint_path)
assert ckpt, "checkpoint path %s not found" % checkpoint_path
global_counter = int(re.findall("-(\d+)$", ckpt.model_checkpoint_path)[0])
saver.restore(sess, ckpt.model_checkpoint_path)
#dqn_agent.global_counter = global_counter

dqn_agent.do_training = False
dqn_agent.do_rollouts = True

save_freq_episodes = 5

print(dqn_agent.get_epsilon())
print(dqn_agent.global_counter)

/home/dahlin/Desktop/Car Racing/car_racing_retry/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dahlin/Desktop/Car Racing/car_racing_retry/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dahlin/Desktop/Car Racing/car_racing_retry/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

In [4]:
now = dt.now()
os.path.join(checkpoint_path,'dqn_rollouts_'+now.strftime('%Y%m%d_%H%M')+'.h5')

'/home/dahlin/Desktop/Unpack/Car Racing Ellipse/Checkpoints/dqn_rollouts_20210520_0009.h5'

In [2]:
now = dt.now()
desired_len = 100000

frames_rollout = []
actions_rollout = []
rewards_rollout = []
num_frames_rollout = []
step_rewards_rollout = []
dones_rollout = []

#total_reward, frames_in_episode, eh = dqn_agent.play_episode(log_data=True)
total_reward, frames_in_episode, actions, frames, step_rewards, dones = dqn_agent.play_episode()
#total_reward, frames_in_episode, actions, frames = dqn_agent.play_episode()
#actions_rollout.extend(eh.actions[0:frames_in_episode])
#frames_rollout.extend(eh.frames[0:frames_in_episode])
actions_rollout.extend(actions)
frames_rollout.extend(frames)
rewards_rollout.append(total_reward)
num_frames_rollout.append(frames_in_episode)
step_rewards_rollout.extend(step_rewards)
dones_rollout.extend(dones)
print('Actions so far : '+str(len(actions_rollout)))
print("episode: %d, reward: %f, length: %d, total steps: %d" %
          (dqn_agent.episode_counter, total_reward, frames_in_episode, dqn_agent.global_counter))

while(len(actions_rollout)<desired_len):
    #total_reward, frames_in_episode, actions, frames = dqn_agent.play_episode()
    total_reward, frames_in_episode, actions, frames, step_rewards, dones = dqn_agent.play_episode()
    #actions_rollout.extend(eh.actions[0:frames_in_episode])
    #frames_rollout.extend(eh.frames[0:frames_in_episode])
    actions_rollout.extend(actions)
    frames_rollout.extend(frames)
    rewards_rollout.append(total_reward)
    num_frames_rollout.append(frames_in_episode)
    step_rewards_rollout.extend(step_rewards)
    dones_rollout.extend(dones)
    print('Actions so far : '+str(len(actions_rollout)))
    print("episode: %d, reward: %f, length: %d, total steps: %d" %
          (dqn_agent.episode_counter, total_reward, frames_in_episode, dqn_agent.global_counter))
    save_cond = dqn_agent.episode_counter % save_freq_episodes == 0
    if save_cond:
        #f = h5py.File('dqn_rollouts'+str(desired_len)+'_frames_only.h5','w')
        #f = h5py.File(checkpoint_path+'/dqn_rollouts.h5','w')
#f = h5py.File('dqn_rollouts'+str(104737)+'.h5','w')
        with h5py.File(os.path.join(checkpoint_path,'dqn_rollouts_'+now.strftime('%Y%m%d_%H%M')+'.h5'),'w') as f:
            f.create_dataset('actions_rollout',data=actions_rollout)
            f.create_dataset('frames_rollout',data=frames_rollout)
            f.create_dataset('rewards_rollout',data=rewards_rollout)
            f.create_dataset('num_frames_rollouts',data=num_frames_erollout)
            f.create_dataset('step_rewards_rollout',data=step_rewards_rollout)
            f.create_dataset('dones_rollout',data=dones_rollout)

k generation: 1420..1777 -> 357-tiles track
Actions so far : 64225
episode: 259, reward: 903.128090, length: 248, total steps: 64225
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 64473
episode: 260, reward: 903.128090, length: 248, total steps: 64473
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 64721
episode: 261, reward: 903.128090, length: 248, total steps: 64721
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 64969
episode: 262, reward: 903.128090, length: 248, total steps: 64969
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 65217
episode: 263, reward: 903.128090, length: 248, total steps: 65217
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 65465
episode: 264, reward: 903.128090, length: 248, total steps: 65465
Track generation: 1420..1777 -> 357-tiles track
Actions so far : 65713
episode: 265, reward: 903.128090, length: 248, total steps: 65713
Track generation: 1420..1777 -> 357-tiles tra

In [3]:
rollout_dict = {}

with h5py.File(os.path.join(checkpoint_path,'dqn_rollouts_'+now.strftime('%Y%m%d_%H%M')+'.h5'),'r') as f:
    for filekey in f.keys():
        print(filekey)
        print(np.shape(f[filekey][:]))
        rollout_dict[filekey] = f[filekey][:]

actions_rollout
(1233,)
dones_rollout
(1233,)
frames_rollout
(1233, 96, 96)
num_frames_rollouts
(5,)
rewards_rollout
(5,)
step_rewards_rollout
(1233,)
